In [ ]:
pip install -U sentence-transformers

In [1]:
# Packages

#pip install -U sentence-transformers
from termcolor import colored
import matplotlib.pyplot as plt
import pickle
import pandas as pd
import numpy as np
import string
import regex as re
import zipfile
import gc
from scipy.stats import boxcox
import sys 
from collections import Counter 
from tqdm import tqdm 
import pandas as pd 
import numpy as np 
import warnings, math
from termcolor import colored
import pickle
import string
from sklearn.metrics.pairwise import cosine_similarity

# for eval
from sklearn.model_selection import train_test_split
import random


# for SVD
from scipy.sparse.linalg import svds
from scipy.sparse import csr_matrix
pd.set_option('display.float_format','{:.5f}'.format)

# for EMB
from sentence_transformers import SentenceTransformer, util
import scipy
from sklearn import preprocessing 

# for TFIDF
from sklearn.feature_extraction.text import TfidfVectorizer
import scipy
from sklearn import preprocessing 

# for creating a network
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
EXT_FEATURES_PATH = 'D:\Papers\Paper 3 - Recommender Systems\Recommender-systems\Files\Graph_based\ext_features_agg.csv'
NEW_DF_PATH = 'D:\Papers\Paper 3 - Recommender Systems\Recommender-systems\Files\Graph_based\Oct_28_df.csv'
PROJECTS_DF_PATH = "D:\Papers\Paper 3 - Recommender Systems\Recommender-systems\Files\projects.csv"


ext_features = pd.read_csv(EXT_FEATURES_PATH)
df = pd.read_csv(NEW_DF_PATH)
projects_df = pd.read_csv(PROJECTS_DF_PATH)


In [4]:
df_train, df_test = train_test_split(df, test_size=0.1, random_state=42)

print('# donations in Train set: ', f"{len(df_train):,}")
print('# donations in Test set: ',  f"{len(df_test):,}")

print('# Donors in Train set: ', f"{len(df_train['Donor ID'].unique()):,}")
print('# Donors in Test set: ',  f"{len(df_test['Donor ID'].unique()):,}")


print('# Donors in both Train and Test sets - the ones we choose for evaluation: ',
      colored(f"{len(df_train[df_train['Donor ID'].isin(df_test['Donor ID'].values.tolist())]['Donor ID'].unique()):,}", 'blue'))



df_train = df_train[df_train['Donor ID'].isin(df_test['Donor ID'].values.tolist())].reset_index(drop=True)
df_test = df_test[df_test['Donor ID'].isin(df_train['Donor ID'].values.tolist())].reset_index(drop=True)

# sum of donation in a grouped by donor id dataset
df_main_donor_index = df.groupby(by = ['Donor ID', 'Project ID']).sum()[['Donation Amount']].reset_index().set_index('Donor ID')
df_train_donor_index = df_train.groupby(by = ['Donor ID', 'Project ID']).sum()[['Donation Amount']].reset_index().set_index('Donor ID')
df_test_donor_index = df_test.groupby(by = ['Donor ID', 'Project ID']).sum()[['Donation Amount']].reset_index().set_index('Donor ID')

# donations in Train set:  43,604
# donations in Test set:  4,845
# Donors in Train set:  790
# Donors in Test set:  617
# Donors in both Train and Test sets - the ones we choose for evaluation:  617


In [5]:
projects_df = projects_df[projects_df['Project ID'].isin(df['Project ID'])].reset_index(drop=True)

project_txt = projects_df.loc[:, 'project_txt']
projects_id = projects_df['Project ID'].tolist()


### Create Embeddings (projects)

In [ ]:
# ----------------- embeddings---------------------#
model = SentenceTransformer('paraphrase-distilroberta-base-v1') # , device= 'cuda'
embeddings = model.encode(project_txt, batch_size=256, show_progress_bar=True)

In [ ]:
with open(EMBEDDING_PATH, 'wb') as f:
    pickle.dump(embeddings, f)

### Create Embeddings (donors profile)

In [5]:
EMBEDDING_PATH = 'D:\Papers\Paper 3 - Recommender Systems\Recommender-systems\Files\Graph_based\Oct_28_project_embeddings.pickle'
project_embeddings_matrix = pd.read_pickle(EMBEDDING_PATH)

In [16]:
# ==================== Embeddings ==================== #

# ----------------------- projects profiles------------------- #
def get_project_profile_emb(project_id: str, project_embeddings_matrix):

    # get the ids
    idx = projects_id.index(project_id)
    project_profile = project_embeddings[idx:idx+1]
    
    return project_profile



def get_projects_profiles_emb(ids: pd.Series, project_embeddings_matrix):


    profiles_list = [get_project_profile_emb(project_id, project_embeddings)[0] for project_id in np.ravel([ids])]
    project_profiles = np.vstack(profiles_list)

    return project_profiles


# ----------------------- Donors profiles------------------- #

def build_donors_profile_emb(donor_id: str, df_train_donor_index: pd.DataFrame):

    # get the id of each person and the projects they
    # donated to
    donations_donor_df = df_train_donor_index.loc[donor_id]


    # get the vectors of projects this person has donated to
    donor_donated_project_intrain_profiles = get_projects_profiles_emb(donations_donor_df['Project ID'], project_embeddings)


    # get the smoothed donated amount as the weight of each project
    donor_project_strengths = np.array(donations_donor_df['Donation Amount']).reshape(-1, 1)


    # multiply the weights and tfidf vectors
    multiplication = np.multiply(donor_donated_project_intrain_profiles, donor_project_strengths)


    # now we normalize the whole vector 
    normalized_donor_preference = preprocessing.normalize(np.sum(multiplication, axis=0).reshape(1, -1))
    


    return normalized_donor_preference


def build_donors_profiles_emb(df_test_donor_index: pd.DataFrame, df_train_donor_index: pd.DataFrame, sample_size: int = None):
    
    # now for all donors we build a profile in a dictionary
    donor_profiles = {}
    donors_in_test_set = df_test_donor_index.index.unique().tolist()[:sample_size]

    for donor_id in tqdm(donors_in_test_set[:sample_size], position=0, leave=True):
        donor_profiles[donor_id] = build_donors_profile_emb(donor_id, df_train_donor_index)

    return donor_profiles

    



In [17]:
donor_profiles_emb = build_donors_profiles_emb(df_test_donor_index, df_train_donor_index)

100%|██████████| 617/617 [00:00<00:00, 1377.75it/s]


In [ ]:
len(donor_profiles_emb['009d5fc7b87883ffad248db5150bf1fc'][0]) == 768

In [ ]:
PROFILE_EMBEDDING_PATH = 'D:\Papers\Paper 3 - Recommender Systems\Recommender-systems\Files\Graph_based\Oct_28_donor_profiles_embeddings.pickle'

with open(PROFILE_EMBEDDING_PATH, 'wb') as f:
    pickle.dump(donor_profiles_emb, f)

### Graph

In [6]:
PROFILE_EMBEDDING_PATH = r'D:\Papers\Paper 3 - Recommender Systems\Recommender-systems\Files\Graph_based\Oct_28_donor_profiles_embeddings.pickle'
PROJECT_EMBEDDING_PATH = r'D:\Papers\Paper 3 - Recommender Systems\Recommender-systems\Files\Graph_based\Oct_28_project_embeddings.pickle'
USER_PROFILE_PATH = r'D:\Papers\Paper 3 - Recommender Systems\Recommender-systems\Files\Graph_based\user_profile.csv'

project_embeddings = pd.read_pickle(PROJECT_EMBEDDING_PATH)
donor_profiles_emb = pd.read_pickle(PROFILE_EMBEDDING_PATH)
user_profile_df = pd.read_csv(USER_PROFILE_PATH)

In [7]:
donors_id = list(donor_profiles_emb.keys())# donor_profiles_emb = user_embeddings

In [8]:
# interaction matrixes for similarity scores-------------------------------------

#project
project_interactions = linear_kernel(project_embeddings_matrix, project_embeddings_matrix)

# donors
donor_embeddings_matrix = np.array([donor_profiles_emb[id][0] for id in donors_id])
donors_interactions  = linear_kernel(donor_embeddings_matrix, donor_embeddings_matrix)


In [9]:
# DONOR - DONOR / PROJ-PROJ edges dict -----------------------------------------------------------------------


def create_edges(ids: list, interaction_matrix: np.array, top_n_similar: int = 20) -> dict:

    """
    output = {'donor_id': [(score, similar donor_id)] }

    """
    edges_dict = {}
    for idx, emb in enumerate(ids):
        similar_indices = interaction_matrix[idx].argsort()[:-top_n_similar:-1]
        
        # idx = index of donor/project
        # i = index of similar donor/project
        # similar_items = list(score, similar donor_id/project_id)
        similar_items = [(interaction_matrix[idx][i], ids[i]) for i in similar_indices]
        edges_dict[ids[idx]] = similar_items

    return edges_dict


donors_edges = create_edges(donors_id, donors_interactions)
project_edges = create_edges(projects_id, project_interactions)

In [10]:
# Graphs -----------------------------------------------------------

# Donor graph---------------------------------------------------------------

class Graph():
    """
    Create the donors'/projects' graph and save their information in nodes.
    """
    
    
    def __init__(self, graph_name: str):
        self.graph = {}
        self.graph_name = graph_name
    
    # function to add new nodes in the graph
    def _create_node(self, node_id: str, node_properties):
        self.graph[node_id] = node_properties 
    
    # function to view the nodes in the graph
    def _view_nodes(self):
        return self.graph
    
    # function to create edges
    def _create_edges(self, node_id, node_edges):
        if node_id in self.graph:
            self.graph[node_id]['edges'] = node_edges

In [11]:
## initialize the donors graph
dg = Graph(graph_name = 'donor')
pg = Graph(graph_name = 'projects')


## NODES ------------------------------------------------------------------------------------------
# donors
for donor_id in donors_id:
    # donor_id = node_id
    node_properties = dict(next(user_profile_df[user_profile_df['Donor ID'] == donor_id].iterrows())[1])
    node_properties['donor_project_emb'] = donor_profiles_emb[donor_id][0]
    
    dg._create_node(donor_id, node_properties)


# projects
for idx, project_id in enumerate(projects_id):
    # donor_id = node_id
    node_properties = dict(next(projects_df[projects_df['Project ID'] == project_id].iterrows())[1])
    node_properties['project_embedding'] = project_embeddings_matrix[idx]

    del node_properties['Project Need Statement']
    #if idx==0: print(node_properties)
    pg._create_node(project_id, node_properties)



# EDGES -----------------------------------------------------------------

def create_edges_in_graph(graph: object, edges: dict, ids:list, top_n_similar: int = 5):

    for id in ids:
        node_edges = edges[id][:top_n_similar]
        graph._create_edges(id, node_edges)

create_edges_in_graph(graph = dg, edges = donors_edges, ids= donors_id, top_n_similar = 5)
create_edges_in_graph(graph = pg, edges = project_edges, ids= projects_id, top_n_similar = 5)


In [12]:
#dg._view_nodes()[donor_id]
#pg._view_nodes()[project_id]

### Graphs Visualization

In [13]:
# Get the D3 host locally. 
from IPython.core.display import display, HTML
from string import Template
import pandas as pd
import json, random

In [14]:
HTML(
    '''
    <style scoped>
    .this_style {
        color: white;
        font: 16px;
    }
    </style>
<h1 class="this_style"> hi there </h1>
    '''
)

In [15]:
len(dg._view_nodes())

610

In [16]:
from IPython.core.display import display, HTML, Javascript
import IPython.display
import json

FOLDER_PATH = r"D:\Papers\Paper 3 - Recommender Systems\Recommender-systems\Files\Graph_based\graph_vis"

nodes = []
links = []

nodes.append({'id' : 'Donors', 'group' : 2, 'size' : 20 })
for key, val in list(dg._view_nodes().items())[:10]:
    if len(val['edges']) == 0:
        continue
    nodes.append({'id' : key, 'group' : 2, 'size' : 15})
    links.append({"source" : "Donors", "target" : key, "value" : 10})
    
    for node in val['edges']:
        nodes.append({'id' : node[1], 'group' : 2, 'size' : 12})
        
        sv = np.log(node[0])
        ew = 10
        if sv > 6:
            ew = 100
        elif sv > 5:
            ew = 20
        elif sv > 4:
            ew = 15
        else:
            ew = 10
                    
        links.append({"source": key, "target": node[1], "value": ew})
doc = {'nodes' : nodes, 'links' : links}
with open(FOLDER_PATH + "\donorg.json", "w") as fout:
    fout.write(json.dumps(doc))
    
    

nodes = []
links = []
nodes.append({'id' : 'Projects', 'group' : 0, 'size' : 20, "title" : "Projects" })
for key, val in list(pg._view_nodes().items())[:10]:
    if len(val['edges']) == 0:
        continue

    nodes.append({'id' : key, 'group' : 0, 'size' : 15})
    links.append({"source" : "Projects","title" : val['Project Title'], "target" : key, "value" : 10})
    for node in val['edges']:
        title = projects_df[projects_df['Project ID'] == node[1]]['Project Title'].iloc(0)[0]
        nodes.append({'id' : node[1], 'group' : 2, 'size' : 12, "title": title})
        links.append({"source": key, "target": node[1], "value": 8})
doc = {'nodes' : nodes, 'links' : links}
with open(FOLDER_PATH + "\projectg.json", "w") as fout:
    fout.write(json.dumps(doc))

In [17]:
HTML('<script src="lib/d3/d3.min.js"></script>')

In [18]:
from IPython.core.display import display, HTML
from string import Template
import json

HTML('')

css_text = '''
'''

js_text_template = Template('''
       var bogoSVG = d3.select("#$bogoanimation") 
          .append("svg")
          .attr("width", 300)
          .attr("height", 300);    

      var data = $python_data ;
       bogoSVG.append("circle")
          .style("stroke", "gray")
          .style("fill", "cyan")
          .attr("r", data[0]['r'])
          .attr("cx", data[0]['cx'])
          .attr("cy", data[0]['cy'])
          .transition()
             .delay(100)
             .duration(10000)  
             .attr("r", 10)
             .attr("cx", data[0]['cx'])
             .style("fill", "blue"); 
''')

html_template = Template('''



''')

js_text = js_text_template.substitute({'python_data': json.dumps([{'r': 130, 'cx': 150, 'cy': 150}]),
                                       'bogoanimation': 'animation'})

HTML(html_template.substitute({'css_text': css_text, 'js_text': js_text}))

In [17]:
#!pip install jupyter_contrib_nbextensions


In [18]:
#!jupyter contrib nbextension install --user

In [19]:
!jupyter nbextension enable codefolding/main

Enabling notebook extension codefolding/main...
      - Validating: ok


In [20]:
%%javascript
require.config({ 
     paths: { 
     d3: 'https://d3js.org/d3.v5.min'
}});

<IPython.core.display.Javascript object>

In [22]:
html7="""
<!DOCTYPE html>
<meta charset="utf-8">
<style>

.links line {
  stroke: #999;
  stroke-opacity: 0.8;
}
.node text {
  pointer-events: none;
  font: 10px sans-serif;
}

.tooldiv {
    display: inline-block;
    width: 120px;
    background-color: white;
    color: #000;
    text-align: center;
    padding: 5px 0;
    border-radius: 6px;
    z-index: 1;
}
.nodes circle {
  stroke: #fff;
  stroke-width: 1.5px;
}

div.tooltip {	
    position: absolute;			
    text-align: center;			
    width: 250px;					
    height: 40px;					
    padding: 2px;				
    font: 12px sans-serif;		
    background: lightsteelblue;	
    border: 0px;		
    border-radius: 8px;			
    pointer-events: none;			
}

</style>
<svg id="donorg" width="860" height="760"></svg>
<h1>hello</h1>
"""

js7="""
require.config({
    paths: {
        d3: 'https://d3js.org/d3.v5.min'
     }
 });
 
require(["d3"], function(d3) {// Dimensions of sunburst.
 console.log(d3.version);
var svg = d3.select("#donorg"),
    width = +svg.attr("width"),
    height = +svg.attr("height");

var color = d3.scaleOrdinal(d3.schemeCategory20);

var simulation = d3.forceSimulation()

    .force("link", d3.forceLink().id(function(d) { return d.id; }).distance(20).strength(1))
    .force("charge", d3.forceManyBody().strength(-155))
    .force("center", d3.forceCenter(width / 2, height / 2));

d3.json("donorg.json", function(error, graph) {
  if (error) throw error;

  var link = svg.append("g")
      .attr("class", "links")
    .selectAll("line")
    .data(graph.links)
    .enter().append("line")
      .attr("stroke-width", function(d) { return Math.sqrt(d.value); });

// Define the div for the tooltip
var div = d3.select("body").append("div")	
    .attr("class", "tooltip")				
    .style("opacity", 0);

  var node = svg.append("g")
      .attr("class", "nodes")
    .selectAll("circle")
    .data(graph.nodes)
    .enter().append("circle")
      .attr("r", function(d) {return d.size})
      .attr("fill", function(d) { return color(d.group); })
      .call(d3.drag()
          .on("start", dragstarted)
          .on("drag", dragged)
          .on("end", dragended)).on("mouseover", function(d) {		
            div.transition()		
                .duration(200)		
                .style("opacity", .9);		
            div.html(d.id)
                .style("left", (d3.event.pageX) + "px")		
                .style("top", (d3.event.pageY - 28) + "px");	
            })					
        .on("mouseout", function(d) {		
            div.transition()		
                .duration(500)		
                .style("opacity", 0);	
        });
          
    

  simulation
      .nodes(graph.nodes)
      .on("tick", ticked);
      

  simulation.force("link")
      .links(graph.links);

  function ticked() {
    link
        .attr("x1", function(d) { return d.source.x; })
        .attr("y1", function(d) { return d.source.y; })
        .attr("x2", function(d) { return d.target.x; })
        .attr("y2", function(d) { return d.target.y; });

    node
        .attr("cx", function(d) { return d.x; })
        .attr("cy", function(d) { return d.y; });
  }
});

function dragstarted(d) {
  if (!d3.event.active) simulation.alphaTarget(0.3).restart();
  d.fx = d.x;
  d.fy = d.y;
}

function dragged(d) {
  d.fx = d3.event.x;
  d.fy = d3.event.y;
}

function dragended(d) {
  if (!d3.event.active) simulation.alphaTarget(0);
  d.fx = null;
  d.fy = null;
}
 });
"""

h = display(HTML(html7))
j = IPython.display.Javascript(js7)
IPython.display.display_javascript(j)

In [23]:
Html_file= open("filename.html","w")
Html_file.write(html7)
Html_file.close()

In [23]:
%%javascript
(function(element) {
    require(['d3'], function(d3) {   
        var data = [1, 2, 4, 8, 16, 8, 4, 2, 1]

        var svg = d3.select(element.get(0)).append('svg')
            .attr('width', 400)
            .attr('height', 200);
        svg.selectAll('circle')
            .data(data)
            .enter()
            .append('circle')
            .attr("cx", function(d, i) {return 40 * (i + 1);})
            .attr("cy", function(d, i) {return 100 + 30 * (i % 3 - 1);})
            .style("fill", "#1570a4")
            .transition().duration(2000)
            .attr("r", function(d) {return 2*d;})
        ;
    })
})(element);

<IPython.core.display.Javascript object>

In [26]:
from string import Template
html_template = Template('''
<style scoped> .bedazzled {color: orange;} </style>
<div id="d3-div-2"></div>
<script>

var size_data = $size_data_python ;

d3.select("#d3-div-2").selectAll('.bedazzled')
    .data(size_data)
    .enter().append('p')
      .attr("class","bedazzled")
      .style("font-size", function(d){ return "" + d + "px";})
      .text("Hello D3 with Python data!");

</script>
''')
HTML(html_template.substitute({'size_data_python': str([15,30,45])}))

### resume graphs